In [ ]:
#Imports
import numpy as np
from scipy.stats import unitary_group
import matplotlib.pyplot as plt
from qiskit import(
    QuantumCircuit
    , execute
    , Aer
    , ClassicalRegister
    , QuantumRegister
)
backend_svec = Aer.get_backend('statevector_simulator')
backend_qasm = Aer.get_backend('qasm_simulator')
from qiskit.quantum_info import partial_trace, DensityMatrix


'''Visualizations'''
from qiskit.visualization import plot_histogram
from qiskit.circuit.random import random_circuit
import matplotlib.pyplot as plt
import time

# Random Circuit Example

In [ ]:
#random 2 qubit circuit 
random_circ=random_circuit(2,2)
print(random_circ)
#print(random_circ.data)
gates=[]
for gate, qargs, cargs in random_circ.data:
    # Each gate is an operation with qargs (quantum arguments, i.e., qubits)
    # and cargs (classical arguments, if any).
    gates.append(f"{gate.name}")
print(gates)

In [ ]:
backend = Aer.get_backend('unitary_simulator')
job = execute(random_circ, backend)
result = job.result()
T_unitary = result.get_unitary(random_circ)
#print(T_unitary.data)
print(np.array2string(T_unitary.data, separator=','))

In [ ]:
RandRho2=qt.rand_dm(2).full()
print(RandRho2)

# 4 Qubit Steerable

This program determines whether a system of 4 qubits is steerable. One can adjust the coefficients of the input state in the first cell and obtain the Fully Entangled Fraction using the function in the third cell. One can also visualize the results by running the fourth cell, which will produce a graph of the maximum FEF vs the number of trial unitaries tested, along with a horizonal line representing the steerability cutoff for this size system. 

In [ ]:
#input (original) rho
coefs4 = {
    "c0000": 1/np.sqrt(2) + 0j, "c0001": 0 + 0j, "c0010": 0 + 0j, "c0011":  + 0j,
    "c0100": 0 + 0j, "c0101": 0 + 0j, "c0110": 0 + 0j, "c0111": 0 + 0j,
    "c1000": 0 + 0j, "c1001": 0 + 0j, "c1010": 0 + 0j, "c1011":0 + 0j,
    "c1100": 0 + 0j, "c1101": 0 + 0j, "c1110": 0 + 0j, "c1111": 1/np.sqrt(2) + 0j
}

PsiKet4 = np.array([
        coefs4["c0000"], coefs4["c0001"], coefs4["c0010"], coefs4["c0011"],
        coefs4["c0100"], coefs4["c0101"], coefs4["c0110"], coefs4["c0111"],
        coefs4["c1000"], coefs4["c1001"], coefs4["c1010"], coefs4["c1011"],
        coefs4["c1100"], coefs4["c1101"], coefs4["c1110"], coefs4["c1111"]
    ]).reshape(-1, 1)
PsiBra4 = PsiKet4.conjugate().T
Rho4 = PsiKet4 @ PsiBra4

In [ ]:
#Trial FullyEntangledFraction for Original Rho
def PosFEF4Rho(Rho):
    PsiEntKet = (1 / 2) * np.array([
        1, 0, 0, 0,
        0, 1, 0, 0,
        0, 0, 1, 0,
        0, 0, 0, 1
    ]).reshape(-1, 1)
    PsiEntBra = PsiEntKet.T
    U = unitary_group.rvs(4)
    I4 = np.identity(4)
    UtensI4 = np.kron(U, I4)
    UtensI4dag = UtensI4.conjugate().T
    return (PsiEntBra @ UtensI4 @ Rho @ UtensI4dag @PsiEntKet).item()

#print(PosFEF4Rho(Rhoo))

In [ ]:
#Maximized Fully Engtangled Fraction for Original Rho
def FullyEntangledFraction4Rho(Rho,trials):
    frac = 0
    i = 0
    while i < trials:
        F=PosFEF4Rho(Rho).real
        if F > frac:
            frac = F
        i+=1
    return frac

RandRho4=qt.rand_dm(16).full()

print(FullyEntangledFraction4Rho(RandRho4,100000))

In [ ]:
#Graph of Fully Entangled Fraction for Original Rho
max_num_trials = 1000000 #change this for your maximum number of trials


xarr = np.linspace(0,max_num_trials,21)
yarr = []


for x in xarr:
    y = FullyEntangledFraction4Rho(Rho4,x)
    yarr.append(y)

# Plot the function
plt.figure(figsize=(8, 6))  # Set the figure size
plt.plot(xarr, yarr, label="maxFEF", color="blue")
plt.axhline(y=0.5854, color='red', linestyle='--', linewidth=1.5, label='Steerable = 0.5854')  # Horizontal line

# Add labels, title, and legend
plt.title("Graph of max Fully Entangled Fraction vs. Trials", fontsize=14)
plt.xlabel("trials", fontsize=12)
plt.ylabel("maxFEF", fontsize=12)
plt.axhline(0, color="black", linewidth=0.8)  # x-axis
plt.axvline(0, color="black", linewidth=0.8)  # y-axis
plt.grid(True)  # Add a grid
plt.legend(fontsize=12)

# Show the plot
plt.show()

# 6 Qubit Steerable

This program determines whether a system of 6 qubits is steerable. One can adjust the coefficients of the input state in the first cell and obtain the Fully Entangled Fraction using the function in the third cell. One can also visualize the results by running the fourth cell, which will produce a graph of the maximum FEF vs the number of trial unitaries tested, along with a horizonal line representing the steerability cutoff for this size system. 

In [ ]:
# Define specific non-zero coefficients for 6 qubits 
non_zero_states6 = {
    0: 1/2 + 0j,    # Corresponds to c000000
    1: 1/2 + 0j,    # Corresponds to c000001
    2: 1/2 + 0j,    # Corresponds to c000010
    63: 1/2 + 0j    # Corresponds to c111111
}


PsiKet6 = np.array([
    non_zero_states6.get(i, 0 + 0j) for i in range(64)
]).reshape(-1, 1)


Rho6 = PsiKet6 @ PsiKet6.conjugate().T

print(Rho6)

In [ ]:
def PosFEF6Rho(Rho_):
    # Define specific non-zero coefficients
    coefficients6 = {
    0: 1/np.sqrt(8) + 0j,    # c000000
    9: 1/np.sqrt(8) + 0j,    # c001001
    18: 1/np.sqrt(8) + 0j,   # c010010
    27: 1/np.sqrt(8) + 0j,   # c011011
    36: 1/np.sqrt(8) + 0j,   # c100100
    45: 1/np.sqrt(8) + 0j,   # c101101
    54: 1/np.sqrt(8) + 0j,   # c110110
    63: 1/np.sqrt(8) + 0j    # c111111
}


    # Create the 64-entry column vector
    PsiKet6ENT = np.array([
    coefficients6.get(i, 0 + 0j) for i in range(64)
    ]).reshape(-1, 1)

    PsiBra6ENT= PsiKet6ENT.conjugate().T
    U = unitary_group.rvs(8)
    UtensI = np.kron(U,np.identity(8))
    return (PsiBra6ENT @ UtensI @ Rho_ @ UtensI.conjugate().T @PsiKet6ENT).item()

print(PosFEF6Rho(Rho6))
    

In [ ]:
#Maximized Fully Engtangled Fraction for Original Rho
def FullyEntangledFraction6Rho(Rho,trials):
    frac = 0
    i = 0
    while i < trials:
        F=PosFEF6Rho(Rho).real
        if F > frac:
            frac = F
        i+=1
    return frac

print(FullyEntangledFraction6Rho(Rho6bell,100000))

In [ ]:
#Graph of Fully Entangled Fraction for Original Rho

max_num_trials = 1000000 #change this for your maximum number of trials

xarr6 = np.linspace(0,max_num_trials,21)
yarr6 = []


for x in xarr6:
    y = FullyEntangledFraction6Rho(Rho6bell,x)
    yarr6.append(y)

# Plot the function
plt.figure(figsize=(8, 6))  # Set the figure size
plt.plot(xarr6, yarr6, label="maxFEF", color="blue")
plt.axhline(y=0.4816, color='red', linestyle='--', linewidth=1.5, label='Steerable = 0.4816')  # Horizontal line

# Add labels, title, and legend
plt.title("Graph of max Fully Entangled Fraction vs. Trials", fontsize=14)
plt.xlabel("trials", fontsize=12)
plt.ylabel("maxFEF", fontsize=12)
plt.axhline(0, color="black", linewidth=0.8)  # x-axis
plt.axvline(0, color="black", linewidth=0.8)  # y-axis
plt.grid(True)  # Add a grid
plt.legend(fontsize=12)

# Show the plot
plt.show()

# 8 Qubit Steerable 

This program determines whether a system of 8 qubits is steerable. One can adjust the coefficients of the input state in the first cell and obtain the Fully Entangled Fraction using the function in the third cell. One can also visualize the results by running the fourth cell, which will produce a graph of the maximum FEF vs the number of trial unitaries tested, along with a horizonal line representing the steerability cutoff for this size system. 

In [ ]:
# Define specific non-zero coefficients
non_zero_states8 = {
    0: 1/np.sqrt(2) + 0j,  # Corresponds to c00000000
    255: 1/np.sqrt(2) + 0j # Corresponds to c11111111
}

# Create the 256-entry column vector
PsiKet8 = np.array([
    non_zero_states8.get(i, 0 + 0j) for i in range(256)
]).reshape(-1, 1)

Rho8=PsiKet8 @ PsiKet8.conjugate().T

#print(Rho8)

In [ ]:
# Obviously Steerable State
ObvSt8= {
    1: 1/4 + 0j,   # Corresponds to c00000001
    18: 1/4 + 0j,  # Corresponds to c00010010
    35: 1/4 + 0j,  # Corresponds to c00100011
    53: 1/4 + 0j,  # Corresponds to c00110100
    69: 1/4 + 0j,  # Corresponds to c01000101
    86: 1/4 + 0j,  # Corresponds to c01010110
    103: 1/4 + 0j, # Corresponds to c01100111
    120: 1/4 + 0j, # Corresponds to c01111000
    137: 1/4 + 0j, # Corresponds to c10001001
    154: 1/4 + 0j, # Corresponds to c10011010
    171: 1/4 + 0j, # Corresponds to c10101011
    188: 1/4 + 0j, # Corresponds to c10111100
    205: 1/4 + 0j, # Corresponds to c11001101
    222: 1/4 + 0j, # Corresponds to c11011110
    239: 1/4 + 0j, # Corresponds to c11101111
    240: 1/4 + 0j  # Corresponds to c11110000
}

# Create the 256-entry column vector
PsiKetObvSt8 = np.array([
    ObvSt8.get(i, 0 + 0j) for i in range(256)
]).reshape(-1, 1)

RhoObvSt8=PsiKetObvSt8 @ PsiKetObvSt8.conjugate().T

#print(Rho8)

In [ ]:
# Fully Entangled Coefficients
coefficients8 = {
    0: 1/4 + 0j,   # Corresponds to c00000000
    17: 1/4 + 0j,  # Corresponds to c00010001
    34: 1/4 + 0j,  # Corresponds to c00100010
    51: 1/4 + 0j,  # Corresponds to c00110011
    68: 1/4 + 0j,  # Corresponds to c01000100
    85: 1/4 + 0j,  # Corresponds to c01010101
    102: 1/4 + 0j, # Corresponds to c01100110
    119: 1/4 + 0j, # Corresponds to c01110111
    136: 1/4 + 0j, # Corresponds to c10001000
    153: 1/4 + 0j, # Corresponds to c10011001
    170: 1/4 + 0j, # Corresponds to c10101010
    187: 1/4 + 0j, # Corresponds to c10111011
    204: 1/4 + 0j, # Corresponds to c11001100
    221: 1/4 + 0j, # Corresponds to c11011101
    238: 1/4 + 0j, # Corresponds to c11101110
    255: 1/4 + 0j  # Corresponds to c11111111
}


# Create the 256-entry column vector
PsiKet8ENT = np.array([
    coefficients8.get(i, 0 + 0j) for i in range(256)
]).reshape(-1, 1)

PsiBra8ENT= PsiKet8ENT.conjugate().T

ppp=PsiKet8ENT@PsiBra8ENT

#print(PsiKet8ENT)

In [ ]:
def PosFEF8Rho(Rho_):
    # Define specific non-zero coefficients
    coefficients8 = {
    0: 1/4 + 0j,   # Corresponds to c00000000
    17: 1/4 + 0j,  # Corresponds to c00010001
    34: 1/4 + 0j,  # Corresponds to c00100010
    51: 1/4 + 0j,  # Corresponds to c00110011
    68: 1/4 + 0j,  # Corresponds to c01000100
    85: 1/4 + 0j,  # Corresponds to c01010101
    102: 1/4 + 0j, # Corresponds to c01100110
    119: 1/4 + 0j, # Corresponds to c01110111
    136: 1/4 + 0j, # Corresponds to c10001000
    153: 1/4 + 0j, # Corresponds to c10011001
    170: 1/4 + 0j, # Corresponds to c10101010
    187: 1/4 + 0j, # Corresponds to c10111011
    204: 1/4 + 0j, # Corresponds to c11001100
    221: 1/4 + 0j, # Corresponds to c11011101
    238: 1/4 + 0j, # Corresponds to c11101110
    255: 1/4 + 0j  # Corresponds to c11111111
    }


    # Create the 256-entry column vector
    PsiKet8ENT = np.array([
    coefficients8.get(i, 0 + 0j) for i in range(256)
    ]).reshape(-1, 1)

    PsiBra8ENT= PsiKet8ENT.conjugate().T
    U = unitary_group.rvs(16)
    UtensI = np.kron(U,np.identity(16))
    return (PsiBra8ENT @ UtensI @ Rho_ @ UtensI.conjugate().T @PsiKet8ENT).item()

print(PosFEF8Rho(Rho8))

In [ ]:
#Maximized Fully Engtangled Fraction for Original Rho
def FullyEntangledFraction8Rho(Rho,trials):
    frac = 0
    i = 0
    while i < trials:
        F=PosFEF8Rho(Rho).real
        if F > frac:
            frac = F
        i+=1
    return frac

print(FullyEntangledFraction8Rho(RhoObvSt8,1000))

In [ ]:
#Graph of Fully Entangled Fraction for Original Rho
max_num_trials = 1000000 #change this for your maximum number of trials


xarr8 = np.linspace(0,max_num_trials,21)
yarr8 = []


for x8 in xarr8:
    y = FullyEntangledFraction8Rho(ppp,x8)
    yarr8.append(y)

# Plot the function
plt.figure(figsize=(8, 6))  # Set the figure size
plt.plot(xarr8, yarr8, label="maxFEF", color="blue")
plt.axhline(y=0.4167, color='red', linestyle='--', linewidth=1.5, label='Steerable = 0.4167')  # Horizontal line

# Add labels, title, and legend
plt.title("Graph of max Fully Entangled Fraction vs. Trials", fontsize=14)
plt.xlabel("trials", fontsize=12)
plt.ylabel("maxFEF", fontsize=12)
plt.axhline(0, color="black", linewidth=0.8)  # x-axis
plt.axvline(0, color="black", linewidth=0.8)  # y-axis
plt.grid(True)  # Add a grid
plt.legend(fontsize=12)

# Show the plot
plt.show()

# 10 Qubit Steerable

This program determines whether a system of 10 qubits is steerable. One can adjust the coefficients of the input state in the first cell and obtain the Fully Entangled Fraction using the function in the third cell. One can also visualize the results by running the fourth cell, which will produce a graph of the maximum FEF vs the number of trial unitaries tested, along with a horizonal line representing the steerability cutoff for this size system. 

In [ ]:
# Define specific non-zero coefficients
non_zero_states10 = {
    0: 1/np.sqrt(2) + 0j,  # Corresponds to c00000000
    255: 1/np.sqrt(2) + 0j # Corresponds to c11111111
}

PsiKet10 = np.array([
    non_zero_states10.get(i, 0 + 0j) for i in range(1024)
]).reshape(-1, 1)

Rho10=PsiKet10 @ PsiKet10.conjugate().T

#print(Rho10)

In [ ]:
def PosFEF10Rho(Rho_):
    # Define specific non-zero coefficients
    coefficients10 = {
    0: 1/np.sqrt(32) + 0j,  # Corresponds to c0000000000
    33: 1/np.sqrt(32) + 0j,  # Corresponds to c0000100001
    66: 1/np.sqrt(32) + 0j,  # Corresponds to c0001000010
    99: 1/np.sqrt(32) + 0j,  # Corresponds to c0001100011
    132: 1/np.sqrt(32) + 0j,  # Corresponds to c0010000100
    165: 1/np.sqrt(32) + 0j,  # Corresponds to c0010100101
    198: 1/np.sqrt(32) + 0j,  # Corresponds to c0011000110
    231: 1/np.sqrt(32) + 0j,  # Corresponds to c0011100111
    264: 1/np.sqrt(32) + 0j,  # Corresponds to c0100001000
    297: 1/np.sqrt(32) + 0j,  # Corresponds to c0100101001
    330: 1/np.sqrt(32) + 0j,  # Corresponds to c0101001010
    363: 1/np.sqrt(32) + 0j,  # Corresponds to c0101101011
    396: 1/np.sqrt(32) + 0j,  # Corresponds to c0110001100
    429: 1/np.sqrt(32) + 0j,  # Corresponds to c0110101101
    462: 1/np.sqrt(32) + 0j,  # Corresponds to c0111001110
    495: 1/np.sqrt(32) + 0j,  # Corresponds to c0111101111
    528: 1/np.sqrt(32) + 0j,  # Corresponds to c1000010000
    561: 1/np.sqrt(32) + 0j,  # Corresponds to c1000110001
    594: 1/np.sqrt(32) + 0j,  # Corresponds to c1001010010
    627: 1/np.sqrt(32) + 0j,  # Corresponds to c1001110011
    660: 1/np.sqrt(32) + 0j,  # Corresponds to c1010010100
    693: 1/np.sqrt(32) + 0j,  # Corresponds to c1010110101
    726: 1/np.sqrt(32) + 0j,  # Corresponds to c1011010110
    759: 1/np.sqrt(32) + 0j,  # Corresponds to c1011110111
    792: 1/np.sqrt(32) + 0j,  # Corresponds to c1100011000
    825: 1/np.sqrt(32) + 0j,  # Corresponds to c1100111001
    858: 1/np.sqrt(32) + 0j,  # Corresponds to c1101011010
    891: 1/np.sqrt(32) + 0j,  # Corresponds to c1101111011
    924: 1/np.sqrt(32) + 0j,  # Corresponds to c1110011100
    957: 1/np.sqrt(32) + 0j,  # Corresponds to c1110111101
    990: 1/np.sqrt(32) + 0j,  # Corresponds to c1111011110
    1023: 1/np.sqrt(32) + 0j  # Corresponds to c1111111111
    }
    # Create the 1024-entry column vector
    PsiKet10ENT = np.array([
    coefficients10.get(i, 0 + 0j) for i in range(1024)
    ]).reshape(-1, 1)

    PsiBra10ENT= PsiKet10ENT.conjugate().T
    U = unitary_group.rvs(32)
    UtensI = np.kron(U,np.identity(32))
    return (PsiBra10ENT @ UtensI @ Rho_ @ UtensI.conjugate().T @PsiKet10ENT).item()

print(PosFEF10Rho(Rho10))

In [ ]:
#Maximized Fully Engtangled Fraction for Original Rho
def FullyEntangledFraction10Rho(Rho,trials):
    frac = 0
    i = 0
    while i < trials:
        F=PosFEF10Rho(Rho).real
        if F > frac:
            frac = F
        i+=1
    return frac

print(FullyEntangledFraction10Rho(Rho10,1000))

In [ ]:
#Graph of Fully Entangled Fraction for Random Rho

max_num_trials = 1000000 #change this for your maximum number of trials

xarr10 = np.linspace(0,max_num_trials,3)
yarr10 = []


for x10 in xarr10:
    y = FullyEntangledFraction10Rho(Rho10,x10)
    yarr10.append(y)

# Plot the function
plt.figure(figsize=(8, 6))  # Set the figure size
plt.plot(xarr10, yarr10, label="maxFEF", color="blue")
plt.axhline(y=0.3714, color='red', linestyle='--', linewidth=1.5, label='Steerable = 0.3714')  # Horizontal line

# Add labels, title, and legend
plt.title("Graph of max Fully Entangled Fraction vs. Trials", fontsize=14)
plt.xlabel("trials", fontsize=12)
plt.ylabel("maxFEF", fontsize=12)
plt.axhline(0, color="black", linewidth=0.8)  # x-axis
plt.axvline(0, color="black", linewidth=0.8)  # y-axis
plt.grid(True)  # Add a grid
plt.legend(fontsize=12)

# Show the plot
plt.show()

# Losing Qubits 4 

In [ ]:
# Starting Intial Qubit

Orig_Coefs4 = {
    "c0000": 1/2 + 0j, "c0001": 0 + 0j, "c0010": 0 + 0j, "c0011": 0 + 0j,
    "c0100": 0 + 0j, "c0101": 1/2 + 0j, "c0110": 0 + 0j, "c0111": 0 + 0j,
    "c1000": 0 + 0j, "c1001": 0 + 0j, "c1010": 1/2 + 0j, "c1011": 0 + 0j,
    "c1100": 0 + 0j, "c1101": 0 + 0j, "c1110": 0 + 0j, "c1111": 1/2 + 0j
}

Orig_PsiKet4 = np.array([
       Orig_Coefs4["c0000"], Orig_Coefs4["c0001"], Orig_Coefs4["c0010"], Orig_Coefs4["c0011"],
       Orig_Coefs4["c0100"], Orig_Coefs4["c0101"], Orig_Coefs4["c0110"], Orig_Coefs4["c0111"],
       Orig_Coefs4["c1000"], Orig_Coefs4["c1001"], Orig_Coefs4["c1010"], Orig_Coefs4["c1011"],
       Orig_Coefs4["c1100"], Orig_Coefs4["c1101"], Orig_Coefs4["c1110"], Orig_Coefs4["c1111"]
    ]).reshape(-1, 1)
    
#beginning 4 qubit Rho
Orig_Rho4= Orig_PsiKet4 @ Orig_PsiKet4.conjugate().T

In [ ]:
#All together in one function
#takes in original Rho, number of circuits and number of trials
#returns dictionary of form {"Orig_FEF4": Orig_FEF4,"TE_FEF4":TE_FEF4,"TE_A0_FEF4": TE_A0_FEF4,"Gates":gates}


def FEFs4(Orig_Rho4,num_circuits,num_trials):
    FEFs=[]
    for i in range(num_circuits):
        #Random Circuit
        random_circ=random_circuit(2,2)
        backend = Aer.get_backend('unitary_simulator')
        job = execute(random_circ, backend)
        result = job.result()
        Rand_Unitary4 = result.get_unitary(random_circ)
        ItensU=np.kron(np.identity(4),Rand_Unitary4)
        #Gates
        gates=[]
        for gate, qargs, cargs in random_circ.data:
            gates.append(f"{gate.name}")
        #TE Rho's
        TE_Rho4 = ItensU @ Orig_Rho4 @ ItensU.conjugate().T
        TE_Rho_lostA3= partial_trace(TE_Rho4,[3]).data
        qubitA_0 = np.array([[1], [0]])
        TE_Rho_A04= np.kron(qubitA_0 @ qubitA_0.conjugate().T,TE_Rho_lostA3)
        #FEFS
        Orig_FEF4 = FullyEntangledFraction4Rho(Orig_Rho4,num_trials)
        TE_FEF4 = FullyEntangledFraction4Rho(TE_Rho4,num_trials)
        TE_A0_FEF4 =FullyEntangledFraction4Rho(TE_Rho_A04,num_trials)
        FEFs.append({"Orig_FEF4": Orig_FEF4,"TE_FEF4":TE_FEF4,"TE_A0_FEF4": TE_A0_FEF4,"Gates":gates})
    return FEFs

print(FEFs4(Orig_Rho4,4,50))

In [ ]:
#Graph of FEF's for original Rho, Time evolved Rho, and Time Evolved Rho that lost qubit A

num_circuits = 20 #change this to the number of circuits you want to test
num_trials= 5000000 #change this to the number of unitaries you want to randomly sample to get FEFs

xarr = np.linspace(0,num_circuits,num_circuits+1)
yarr = []
Orig4_yarr = []
TE4_yarr = []
TE_A04_yarr = []

for x in xarr:
    y = FEFs4(Orig_Rho4,len(xarr),num_trials)[int(x)]
    yarr.append(y)

for x in xarr:
    y = yarr[int(x)]["Orig_FEF4"]
    Orig4_yarr.append(y)

for x in xarr:
    y = yarr[int(x)]["TE_FEF4"]
    TE4_yarr.append(y)
    
for x in xarr:
    y = yarr[int(x)]["TE_A0_FEF4"]
    TE_A04_yarr.append(y)


# Plot the function
plt.figure(figsize=(8, 6))  # Set the figure size
plt.scatter(xarr, Orig4_yarr, label="OrigmaxFEF", color="blue")
plt.scatter(xarr, TE4_yarr, label="TEmaxFEF", color="navy")
plt.scatter(xarr, TE_A04_yarr, label="TE_A04maxFEF", color="forestgreen")
plt.axhline(y=0.5854, color='mediumpurple', linestyle='--', linewidth=1.5, label='Steerable > 0.5854')  # Horizontal line
# Add labels, title, and legend
plt.title("Graph of max Fully Entangled Fraction for Different Random Circuits", fontsize=14)
plt.xlabel("trials", fontsize=12)
plt.ylabel("maxFEF", fontsize=12)
plt.axhline(0, color="black", linewidth=0.8)  # x-axis
plt.axvline(0, color="black", linewidth=0.8)  # y-axis
plt.grid(True)  # Add a grid
plt.legend(fontsize=12)

# Show the plot
plt.show()

# Losing Qubits 8

In [ ]:
# Starting Intial Qubit

Orig_Coefs8 = {
    0: 1/4 + 0j,   # Corresponds to c00000000
    17: 1/4 + 0j,  # Corresponds to c00010001
    34: 1/4 + 0j,  # Corresponds to c00100010
    51: 1/4 + 0j,  # Corresponds to c00110011
    68: 1/4 + 0j,  # Corresponds to c01000100
    85: 1/4 + 0j,  # Corresponds to c01010101
    102: 1/4 + 0j, # Corresponds to c01100110
    119: 1/4 + 0j, # Corresponds to c01110111
    136: 1/4 + 0j, # Corresponds to c10001000
    153: 1/4 + 0j, # Corresponds to c10011001
    170: 1/4 + 0j, # Corresponds to c10101010
    187: 1/4 + 0j, # Corresponds to c10111011
    204: 1/4 + 0j, # Corresponds to c11001100
    221: 1/4 + 0j, # Corresponds to c11011101
    238: 1/4 + 0j, # Corresponds to c11101110
    255: 1/4 + 0j  # Corresponds to c11111111
}


# Create the 256-entry column vector
Orig_PsiKet8 = np.array([
    Orig_Coefs8.get(i, 0 + 0j) for i in range(256)
]).reshape(-1, 1)

Orig_Rho8= Orig_PsiKet8 @ Orig_PsiKet8.conjugate().T

print(PsiKet8ENT)

In [ ]:
# Put it all Together
#takes in original Rho, number of circuits and number of trials
#returns dictionary of form {"Orig_FEF8": Orig_FEF8,"TE_FEF8":TE_FEF8,"TE_A0_FEF8": TE_A0_FEF8,
#                           "TE_AB0_FEF8": TE_AB0_FEF8,"TE_ABC0_FEF8": TE_ABC0_FEF8,"Gates":gates}


def FEFs8(Orig_Rho8,num_circuits,num_trials):
    FEFs=[]
    for i in range(num_circuits):
        random_circ=random_circuit(4,4)
        backend = Aer.get_backend('unitary_simulator')
        job = execute(random_circ, backend)
        result = job.result()
        Rand_Unitary8 = result.get_unitary(random_circ)#random unitary 
        ItensU8=np.kron(np.identity(16),Rand_Unitary8)
        #Gates
        gates=[]
        for gate, qargs, cargs in random_circ.data:
            gates.append(f"{gate.name}")
        #TE Rho's
        TE_Rho8= ItensU8 @ Orig_Rho8 @ ItensU8.conjugate().T
        qubit0 = np.array([[1], [0]])
        qubit0Rho=qubit0 @ qubit0.conjugate().T
        TE_Rho_lostA7= partial_trace(TE_Rho8,[7]).data
        TE_Rho_lostAB6= partial_trace(TE_Rho_lostA7,[6]).data
        TE_Rho_lostABC5= partial_trace(TE_Rho_lostAB6,[5]).data
        TE_Rho_A08= np.kron(qubit0Rho,TE_Rho_lostA7)
        TE_Rho_AB08= np.kron(np.kron(qubit0Rho,qubit0Rho),TE_Rho_lostAB6)
        TE_Rho_ABC08= np.kron(np.kron(np.kron(qubit0Rho,qubit0Rho),qubit0Rho),TE_Rho_lostABC5)
        #FEFS
        Orig_FEF8 = FullyEntangledFraction8Rho(Orig_Rho8,num_trials)
        TE_FEF8 = FullyEntangledFraction8Rho(TE_Rho8,num_trials)
        TE_A0_FEF8 =FullyEntangledFraction8Rho(TE_Rho_A08,num_trials)
        TE_AB0_FEF8 =FullyEntangledFraction8Rho(TE_Rho_AB08,num_trials)
        TE_ABC0_FEF8 =FullyEntangledFraction8Rho(TE_Rho_ABC08,num_trials)
        FEFs.append({"Orig_FEF8": Orig_FEF8,"TE_FEF8":TE_FEF8,"TE_A0_FEF8": TE_A0_FEF8,"TE_AB0_FEF8": TE_AB0_FEF8,"TE_ABC0_FEF8": TE_ABC0_FEF8,"Gates":gates})
    return FEFs

print(FEFs8(Orig_Rho8,4,50))


In [ ]:
##Graph of FEF's for original Rho, Time evolved Rho, Time Evolved Rho that lost qubit A, 
#Time Evolved Rho that lost qubit A and B, and Time Evolved Rho that lost qubit A and B and C

num_circuits = 20 #change this to the number of circuits you want to test
num_trials= 5000000 #change this to the number of unitaries you want to randomly sample to get FEFs

xarr = np.linspace(0,num_circuits,num_circuits+1)
yarr = []
Orig8_yarr = []
TE8_yarr = []
TE_A08_yarr = []
TE_AB08_yarr = []
TE_ABC08_yarr = []


for x in xarr:
    y = FEFs8(Orig_Rho8,len(xarr),num_trials)[int(x)]
    yarr.append(y)

for x in xarr:
    y = yarr[int(x)]["Orig_FEF8"]
    Orig8_yarr.append(y)

for x in xarr:
    y = yarr[int(x)]["TE_FEF8"]
    TE8_yarr.append(y)
    
for x in xarr:
    y = yarr[int(x)]["TE_A0_FEF8"]
    TE_A08_yarr.append(y)

for x in xarr:
    y = yarr[int(x)]["TE_A0_FEF8"]
    TE_AB08_yarr.append(y)

for x in xarr:
    y = yarr[int(x)]["TE_A0_FEF8"]
    TE_ABC08_yarr.append(y)


# Plot the function
plt.figure(figsize=(8, 6))  # Set the figure size
plt.scatter(xarr, Orig8_yarr, label="OrigmaxFEF", color="blue")
plt.scatter(xarr, TE8_yarr, label="TEmaxFEF", color="navy")
plt.scatter(xarr, TE_A08_yarr, label="TE_A04maxFEF", color="forestgreen")
plt.scatter(xarr, TE_AB08_yarr, label="TE_AB04maxFEF", color="green")
plt.scatter(xarr, TE_ABC08_yarr, label="TE_ABC04maxFEF", color="lightgreen")



plt.axhline(y=0.4167, color='mediumpurple', linestyle='--', linewidth=1.5, label='Steerable > 0.4167')  # Horizontal line
# Add labels, title, and legend
plt.title("Graph of max Fully Entangled Fraction for Different Random Circuits", fontsize=14)
plt.xlabel("trials", fontsize=12)
plt.ylabel("maxFEF", fontsize=12)
plt.axhline(0, color="black", linewidth=0.8)  # x-axis
plt.axvline(0, color="black", linewidth=0.8)  # y-axis
plt.grid(True)  # Add a grid
plt.legend(fontsize=12)

# Show the plot
plt.show()

# Losing Qubits 10

In [ ]:
# Starting Intial Qubit
Orig_Coefs10 = {
    0: 1/np.sqrt(32) + 0j,  # Corresponds to c0000000000
    33: 1/np.sqrt(32) + 0j,  # Corresponds to c0000100001
    66: 1/np.sqrt(32) + 0j,  # Corresponds to c0001000010
    99: 1/np.sqrt(32) + 0j,  # Corresponds to c0001100011
    132: 1/np.sqrt(32) + 0j,  # Corresponds to c0010000100
    165: 1/np.sqrt(32) + 0j,  # Corresponds to c0010100101
    198: 1/np.sqrt(32) + 0j,  # Corresponds to c0011000110
    231: 1/np.sqrt(32) + 0j,  # Corresponds to c0011100111
    264: 1/np.sqrt(32) + 0j,  # Corresponds to c0100001000
    297: 1/np.sqrt(32) + 0j,  # Corresponds to c0100101001
    330: 1/np.sqrt(32) + 0j,  # Corresponds to c0101001010
    363: 1/np.sqrt(32) + 0j,  # Corresponds to c0101101011
    396: 1/np.sqrt(32) + 0j,  # Corresponds to c0110001100
    429: 1/np.sqrt(32) + 0j,  # Corresponds to c0110101101
    462: 1/np.sqrt(32) + 0j,  # Corresponds to c0111001110
    495: 1/np.sqrt(32) + 0j,  # Corresponds to c0111101111
    528: 1/np.sqrt(32) + 0j,  # Corresponds to c1000010000
    561: 1/np.sqrt(32) + 0j,  # Corresponds to c1000110001
    594: 1/np.sqrt(32) + 0j,  # Corresponds to c1001010010
    627: 1/np.sqrt(32) + 0j,  # Corresponds to c1001110011
    660: 1/np.sqrt(32) + 0j,  # Corresponds to c1010010100
    693: 1/np.sqrt(32) + 0j,  # Corresponds to c1010110101
    726: 1/np.sqrt(32) + 0j,  # Corresponds to c1011010110
    759: 1/np.sqrt(32) + 0j,  # Corresponds to c1011110111
    792: 1/np.sqrt(32) + 0j,  # Corresponds to c1100011000
    825: 1/np.sqrt(32) + 0j,  # Corresponds to c1100111001
    858: 1/np.sqrt(32) + 0j,  # Corresponds to c1101011010
    891: 1/np.sqrt(32) + 0j,  # Corresponds to c1101111011
    924: 1/np.sqrt(32) + 0j,  # Corresponds to c1110011100
    957: 1/np.sqrt(32) + 0j,  # Corresponds to c1110111101
    990: 1/np.sqrt(32) + 0j,  # Corresponds to c1111011110
    1023: 1/np.sqrt(32) + 0j  # Corresponds to c1111111111

}


# Create the 256-entry column vector
Orig_Ket10 = np.array([
    Orig_Coefs10.get(i, 0 + 0j) for i in range(1024)
]).reshape(-1, 1)

Orig_Rho10= Orig_Ket10 @ Orig_Ket10.conjugate().T

print(Orig_Rho10.shape)

In [ ]:
# Put it all Together
#takes in original Rho, number of circuits and number of trials
#returns dictionary of form {"Orig_FEF10": Orig_FEF10,"TE_FEF10":TE_FEF10,"TE_A0_FEF10": TE_A0_FEF10,
#                           "TE_AB0_FEF10": TE_AB0_FEF10,"TE_ABC0_FEF10": TE_ABC0_FEF10,"TE_ABCD0_FEF10": TE_ABCD0_FEF10,"Gates":gates}


def FEFs10(Orig_Rho10,num_circuits,num_trials):
    FEFs=[]
    for i in range(num_circuits):
        random_circ=random_circuit(5,5)
        backend = Aer.get_backend('unitary_simulator')
        job = execute(random_circ, backend)
        result = job.result()
        Rand_Unitary10 = result.get_unitary(random_circ)#random unitary 
        ItensU10=np.kron(np.identity(32),Rand_Unitary10)
        #Gates
        gates=[]
        for gate, qargs, cargs in random_circ.data:
            gates.append(f"{gate.name}")
        #TE Rho's
        TE_Rho10= ItensU10 @ Orig_Rho10 @ ItensU10.conjugate().T
        qubit0 = np.array([[1], [0]])
        qubit0Rho=qubit0 @ qubit0.conjugate().T
        TE_Rho_lostA9= partial_trace(TE_Rho10,[9]).data
        TE_Rho_lostAB8= partial_trace(TE_Rho_lostA9,[8]).data
        TE_Rho_lostABC7= partial_trace(TE_Rho_lostAB8,[7]).data
        TE_Rho_lostABCD6= partial_trace(TE_Rho_lostABC7,[6]).data
        TE_Rho_A010= np.kron(qubit0Rho,TE_Rho_lostA9)
        TE_Rho_AB010= np.kron(np.kron(qubit0Rho,qubit0Rho),TE_Rho_lostAB8)
        TE_Rho_ABC010= np.kron(np.kron(np.kron(qubit0Rho,qubit0Rho),qubit0Rho),TE_Rho_lostABC7)
        TE_Rho_ABCD010= np.kron(np.kron(np.kron(np.kron(qubit0Rho,qubit0Rho),qubit0Rho),qubit0Rho),TE_Rho_lostABCD6)

        #FEFS
        Orig_FEF10 = FullyEntangledFraction10Rho(Orig_Rho10,num_trials)
        TE_FEF10 = FullyEntangledFraction10Rho(TE_Rho10,num_trials)
        TE_A0_FEF10 =FullyEntangledFraction10Rho(TE_Rho_A010,num_trials)
        TE_AB0_FEF10 =FullyEntangledFraction10Rho(TE_Rho_AB010,num_trials)
        TE_ABC0_FEF10 =FullyEntangledFraction10Rho(TE_Rho_ABC010,num_trials)
        TE_ABCD0_FEF10 =FullyEntangledFraction10Rho(TE_Rho_ABCD010,num_trials)
        FEFs.append({"Orig_FEF10": Orig_FEF10,"TE_FEF10":TE_FEF10,"TE_A0_FEF10": TE_A0_FEF10,"TE_AB0_FEF10": TE_AB0_FEF10,"TE_ABC0_FEF10": TE_ABC0_FEF10,"TE_ABCD0_FEF10": TE_ABCD0_FEF10,"Gates":gates})
    return FEFs

print(FEFs10(Orig_Rho10,4,50))


In [ ]:
##Graph of FEF's for original Rho, Time evolved Rho, Time Evolved Rho that lost qubit A, 
#Time Evolved Rho that lost qubit A and B, Time Evolved Rho that lost qubit A and B and C, 
#and Time Evolved Rho that lost qubit A and B and C and D

num_circuits = 20 #change this to the number of circuits you want to test
num_trials= 5000000 #change this to the number of unitaries you want to randomly sample to get FEFs


xarr = np.linspace(0,num_circuits,num_circuits+1)
yarr = []
Orig10_yarr = []
TE10_yarr = []
TE_A010_yarr = []
TE_AB010_yarr = []
TE_ABC010_yarr = []
TE_ABCD010_yarr = []



for x in xarr:
    y = FEFs10(Orig_Rho10,len(xarr),num_trials)[int(x)]
    yarr.append(y)

for x in xarr:
    y = yarr[int(x)]["Orig_FEF10"]
    Orig10_yarr.append(y)

for x in xarr:
    y = yarr[int(x)]["TE_FEF10"]
    TE10_yarr.append(y)
    
for x in xarr:
    y = yarr[int(x)]["TE_A0_FEF10"]
    TE_A010_yarr.append(y)

for x in xarr:
    y = yarr[int(x)]["TE_AB0_FEF10"]
    TE_AB010_yarr.append(y)

for x in xarr:
    y = yarr[int(x)]["TE_ABC0_FEF10"]
    TE_ABC010_yarr.append(y)

for x in xarr:
    y = yarr[int(x)]["TE_ABCD0_FEF10"]
    TE_ABCD010_yarr.append(y)


# Plot the function
plt.figure(figsize=(8, 6))  # Set the figure size
plt.scatter(xarr, Orig10_yarr, label="OrigmaxFEF", color="blue")
plt.scatter(xarr, TE10_yarr, label="TEmaxFEF", color="navy")
plt.scatter(xarr, TE_A010_yarr, label="TE_A010maxFEF", color="forestgreen")
plt.scatter(xarr, TE_AB010_yarr, label="TE_AB010maxFEF", color="green")
plt.scatter(xarr, TE_ABC010_yarr, label="TE_ABC010maxFEF", color="lightgreen")
plt.scatter(xarr, TE_ABCD010_yarr, label="TE_ABCD010maxFEF", color="limegreen")




plt.axhline(y=0.3714, color='mediumpurple', linestyle='--', linewidth=1.5, label='Steerable > 0.3714')  # Horizontal line
# Add labels, title, and legend
plt.title("Graph of max Fully Entangled Fraction for Different Random Circuits", fontsize=14)
plt.xlabel("trials", fontsize=12)
plt.ylabel("maxFEF", fontsize=12)
plt.axhline(0, color="black", linewidth=0.8)  # x-axis
plt.axvline(0, color="black", linewidth=0.8)  # y-axis
plt.grid(True)  # Add a grid
plt.legend(fontsize=12)

# Show the plot
plt.show()